In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
import numpy as np

# Load Foil Data

In [ ]:
from ICARUS.computation.solvers.XFLR5.polars import read_polars_2d
from ICARUS.database import DB
from ICARUS.core.struct import Struct
from ICARUS.database import EXTERNAL_DB

# Get Plane

In [ ]:
# from vehicles.Planes.hermes import hermes
# airplane = hermes("Hermes")

# from ICARUS.computation.solvers.XFLR5.parser import parse_xfl_project
# filename: str = f"{EXTERNAL_DB}/plane_1.xml"
# airplane = parse_xfl_project(filename)
# airplane.name = "Embraer_E190_Cruise"
# airplane.visualize()


# airplane = DB.vehicles_db.planes['hermes_3']
# airplane.save()

In [ ]:
from Planes.hermes import hermes
airplane = hermes("hermes")

# Import Environment and Set State

In [ ]:
from ICARUS.environment.definition import EARTH_ISA
from ICARUS.flight_dynamics.state import State

u_freestream = 20
unstick = State(
    name= "unstick",
    airplane= airplane,
    environment= EARTH_ISA,
    u_freestream= u_freestream
)

# Get Solver

In [ ]:
from ICARUS.computation.solvers.GenuVP.gnvp3 import GenuVP3
gnvp3 = GenuVP3()

from ICARUS.computation.solvers.GenuVP.gnvp7 import GenuVP7
gnvp7 = GenuVP7()

from ICARUS.computation.solvers.Icarus_LSPT.wing_lspt import LSPT
lspt = LSPT()

# print(lspt)
print(gnvp3)
# print(gnvp7)

## AoA Run

In [ ]:
analysis: str = gnvp3.get_analyses_names()[0]
print(f"Selecting Analysis: {analysis}")
gnvp3.select_analysis(analysis)

In [ ]:
options: Struct = gnvp3.get_analysis_options(verbose=True)
solver_parameters: Struct = gnvp3.get_solver_parameters(verbose=True)

In [ ]:
AoAmin = -6
AoAmax = 8
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA, dtype= float)
# airplane.define_dynamic_pressure(u_freestream, EARTH.air_density)

options.plane         = airplane
options.state         = unstick
options.solver2D      = 'Xfoil'
options.maxiter       = 100
options.timestep      = 0.001
options.angles        = angles

solver_parameters.Use_Grid = 0

gnvp3.define_analysis(options, solver_parameters)
gnvp3.print_analysis_options()
gnvp3.execute(parallel = False)
polars = gnvp3.get_results()
airplane.save()

# Dynamics

### Define and Trim Plane

In [ ]:
from ICARUS.flight_dynamics.state import State
if isinstance(polars,int):
    raise Exception("No Polars")
unstick = State("Unstick",airplane,environment= EARTH_ISA, u_freestream= u_freestream)
unstick.add_polar(polars)

### Pertrubations

In [ ]:
unstick.add_all_pertrubations("Central")
unstick.get_pertrub()

In [ ]:
analysis = gnvp3.get_analyses_names(verbose=True)[1] # ANGLES PARALLEL
print(f"Selecting Analysis: {analysis}")
gnvp3.select_analysis(analysis)
options = gnvp3.get_analysis_options(verbose=True)

In [ ]:
options.plane         = airplane
options.state         = unstick
options.solver2D      = 'XFLR'
options.maxiter       = 200
options.timestep      = 5e-3

gnvp3.print_analysis_options()
gnvp3.define_analysis(options, solver_parameters)
gnvp3.execute(parallel=True)
gnvp3.get_results()